In [2]:
import pandas as pd
import numpy as np 
import nltk
import itertools
from nltk.tokenize import sent_tokenize, word_tokenize
import scipy 
from scipy import spatial
import re
from textblob import TextBlob
sw = nltk.corpus.stopwords.words('english')
import gensim
from gensim.models import Word2Vec
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
import string
from nltk.util import ngrams
from textblob import TextBlob
from textblob import Word
from nltk.stem.snowball import SnowballStemmer
sttemmer = SnowballStemmer("english")
from nltk.stem import PorterStemmer
st = PorterStemmer()
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
import gensim
from gensim.models import Word2Vec
from sklearn.cluster import KMeans

In [3]:
data = pd.read_csv("../TED_data.csv")

In [4]:
data

,Unnamed: 0,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,...,film_date.minute,film_date.second,published_date.year,published_date.month,published_date.day_of_month,published_date.day_of_week,published_date.hour,published_date.minute,published_date.second,transcript
0,0,4553,Sir Ken Robinson makes an entertaining and pro...,1164,TED2006,2006-02-25 00:00:00,60,Ken Robinson,Ken Robinson: Do schools kill creativity?,1,...,0,0,2006,6,27,1,0,11,0,Good morning. How are you?(Laughter)It's been ...
1,1,265,With the same humor and humanity he exuded in ...,977,TED2006,2006-02-25 00:00:00,43,Al Gore,Al Gore: Averting the climate crisis,1,...,0,0,2006,6,27,1,0,11,0,"Thank you so much, Chris. And it's truly a gre..."
2,2,124,New York Times columnist David Pogue takes aim...,1286,TED2006,2006-02-24 00:00:00,26,David Pogue,David Pogue: Simplicity sells,1,...,0,0,2006,6,27,1,0,11,0,"(Music: ""The Sound of Silence,"" Simon & Garfun..."
3,3,200,"In an emotionally charged talk, MacArthur-winn...",1116,TED2006,2006-02-26 00:00:00,35,Majora Carter,Majora Carter: Greening the ghetto,1,...,0,0,2006,6,27,1,0,11,0,If you're here today — and I'm very happy that...
4,4,593,You've never seen data presented like this. Wi...,1190,TED2006,2006-02-22 00:00:00,48,Hans Rosling,Hans Rosling: The best stats you've ever seen,1,...,0,0,2006,6,27,1,20,38,0,"About 10 years ago, I took on the task to teac..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2545,2548,17,"Between 2008 and 2016, the United States depor...",476,TED2017,2017-06-06 00:00:00,4,Duarte Geraldino,Duarte Geraldino: What we're missing in the de...,1,...,0,0,2017,9,19,1,20,0,16,"So, Ma was trying to explain something to me a..."
2546,2549,6,How can you study Mars without a spaceship? He...,290,TED2017,2017-04-24 00:00:00,3,Armando Azua-Bustos,Armando Azua-Bustos: The most Martian place on...,1,...,0,0,2017,9,20,2,15,2,17,This is a picture of a sunset on Mars taken by...
2547,2550,10,Science fiction visions of the future show us ...,651,TED2017,2017-04-24 00:00:00,1,Radhika Nagpal,Radhika Nagpal: What intelligent machines can ...,1,...,0,0,2017,9,21,3,15,1,35,"In my early days as a graduate student, I went..."
2548,2551,32,In an unmissable talk about race and politics ...,1100,TEDxMileHigh,2017-07-08 00:00:00,1,Theo E.J. Wilson,Theo E.J. Wilson: A black man goes undercover ...,1,...,0,0,2017,9,21,3,20,0,42,I took a cell phone and accidentally made myse...


# Converting the columns into string type

In [5]:
data = data[['title','transcript']].astype(str)
data.head()

,title,transcript
0,Do schools kill creativity?,Good morning. How are you?(Laughter)It's been ...
1,Averting the climate crisis,"Thank you so much, Chris. And it's truly a gre..."
2,Simplicity sells,"(Music: ""The Sound of Silence,"" Simon & Garfun..."
3,Greening the ghetto,If you're here today — and I'm very happy that...
4,The best stats you've ever seen,"About 10 years ago, I took on the task to teac..."


# Text preprocessing

In [6]:
import nltk
nltk.download("stopwords")
sw = nltk.corpus.stopwords.words("english")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sharm\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
def  text_processing(df):
    """""=== Lower case ==="""
    df['transcript'] = df['transcript'].apply(lambda x: " ".join(x.lower() for x in x.split()))
    
    '''=== Removal of stop words ==='''
    #stopsw = stopwords.words('english')
    df['transcript'] = df['transcript'].apply(lambda x: " ".join(x for x in x.split()if x not in sw))
    
    '''=== Spelling Correction === '''
    #df['transcript'].apply(lambda x: str(TextBlob(x).correct()))
    
    '''=== Removal of Punctuation ==='''
    df['transcript'] = df['transcript'].str.replace('[^\w\s]', '')  

    '''=== Removal of Numeric ==='''
    df['transcript'] = df['transcript'].str.replace('[0-9]', '')
    
    return df

In [8]:
data = text_processing(data)
print(data)

C:\Users\sharm\AppData\Local\Temp\ipykernel_2352\3582746035.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  df['transcript'] = df['transcript'].str.replace('[^\w\s]', '')
C:\Users\sharm\AppData\Local\Temp\ipykernel_2352\3582746035.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  df['transcript'] = df['transcript'].str.replace('[0-9]', '')


                                                  title  \
0                           Do schools kill creativity?   
1                           Averting the climate crisis   
2                                      Simplicity sells   
3                                   Greening the ghetto   
4                       The best stats you've ever seen   
...                                                 ...   
2545  What we're missing in the debate about immigra...   
2546                    The most Martian place on Earth   
2547  What intelligent machines can learn from a sch...   
2548       A black man goes undercover in the alt-right   
2549  How a video game might help us build better ci...   

                                             transcript  
0     good morning youlaughterits great it ive blown...  
1     thank much chris truly great honor opportunity...  
2     music the sound silence simon  garfunkelhello ...  
3     today  im happy  heard sustainable development...  
4

In [10]:
data['transcript'] = data['transcript'].apply(lambda a: " ".join([st.stem(x) for x in a.split()]))

# converting text to features using count vectorizer

In [12]:
cv =CountVectorizer()
cv.fit(data['transcript'])
# Transform the training data using count vectorizer object
cv_tedfeatures = cv.transform(data['transcript'])


In [13]:
#word level TF-IDF
tv = TfidfVectorizer()
tv.fit(data['transcript'])
tv_tedfeatures =  tv.transform(data['transcript'])


# Loading Pretrained Word2vec

In [14]:
#Load 
m1=gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

# Function to get the embeddings
def get_embedding (x, out=False):
    if x in m1.wv.vocab:
        if out == True:
            return m1.wv.vocab[x]
        else:
            return m1[x]
    else:
        return np.zeros(300)

FileNotFoundError: [Errno 2] No such file or directory: 'GoogleNews-vectors-negative300.bin'